#**Implementasi**

In [ ]:
import time
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os
import tenseal as ts

# AES-GCM Encryption and Decryption
def aes_gcm_encrypt_decrypt(plaintext):
    # Generate key and nonce
    password = b"password"
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    nonce = os.urandom(12)

    # Encrypt
    aes_gcm_encrypt_start = time.time()
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(nonce),
        backend=default_backend()
    ).encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    aes_gcm_encrypt_end = time.time()

    # Decrypt
    aes_gcm_decrypt_start = time.time()
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(nonce, encryptor.tag),
        backend=default_backend()
    ).decryptor()
    decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
    aes_gcm_decrypt_end = time.time()

    return aes_gcm_encrypt_end - aes_gcm_encrypt_start, aes_gcm_decrypt_end - aes_gcm_decrypt_start

# FHE Encryption and Decryption using TenSEAL
def fhe_encrypt_decrypt(plaintext):
    # Setup TenSEAL context
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.global_scale = 2**40
    context.generate_galois_keys()

    # Encode plaintext
    plain_vector = [ord(c) for c in plaintext]

    # Encrypt
    fhe_encrypt_start = time.time()
    encrypted = ts.ckks_vector(context, plain_vector)
    fhe_encrypt_end = time.time()

    # Decrypt
    fhe_decrypt_start = time.time()
    decrypted = encrypted.decrypt()
    fhe_decrypt_end = time.time()

    return fhe_encrypt_end - fhe_encrypt_start, fhe_decrypt_end - fhe_decrypt_start

plaintext = "kripto_kapselku"

aes_gcm_enc_time, aes_gcm_dec_time = aes_gcm_encrypt_decrypt(plaintext)
print(f"AES-GCM Encryption time: {aes_gcm_enc_time} seconds")
print(f"AES-GCM Decryption time: {aes_gcm_dec_time} seconds")

fhe_enc_time, fhe_dec_time = fhe_encrypt_decrypt(plaintext)
print(f"FHE Encryption time: {fhe_enc_time} seconds")
print(f"FHE Decryption time: {fhe_dec_time} seconds")


AES-GCM Encryption time: 0.0015723705291748047 seconds
AES-GCM Decryption time: 0.0005505084991455078 seconds
FHE Encryption time: 0.014245986938476562 seconds
FHE Decryption time: 0.00263214111328125 seconds
